Two step workflow:
- get data from MySQL into our HDFS database using Sqoop
- Use Hive to query on the data 

Reminder on how to log in to MySQL using sandbox:
```
mysql -u root -p <password>
```
make sure your database movielens is there and you have all permissions, if not please refer to prev notebook of MySQL

Now get hive script oldmovies.sql:
```
DROP TABLE movies;
CREATE EXTERNAL TABLE movies (movie_id INT, title STRING, release DATE) ROW FORM AT DELIMITED FIELDS TERMINTED BY ',' LOCATION '/user/maria_dev/movies/';
INSERT OVERWRITE DIRECTORY '${OUTPUT}' SELECT * FROM movies WHERE release < '1940-01-01' ORDER BY release;
```
Remember EXTERNAL means it does not create a copy of it, OUTPUT will be set up in the properties file in Oozie job

Below is the Oozie workflow xml file:

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<workflow-app xmlns="uri:oozie:workflow:0.2" name="old-movies">
    <start to="sqoop-node"/>
 
    <action name="sqoop-node">
        <sqoop xmlns="uri:oozie:sqoop-action:0.2">
            <job-tracker>${jobTracker}</job-tracker>
            <name-node>${nameNode}</name-node>
            <prepare> #run before starting the job
                <delete path="${nameNode}/user/maria_dev/movies"/>
            </prepare>
 
            <configuration>
                <property>
                    <name>mapred.job.queue.name</name>
                    <value>${queueName}</value>
                </property>
            </configuration>
            <command>import --connect jdbc:mysql://localhost/movielens --driver com.mysql.jdbc.Driver --table movies -m 1</command>
        </sqoop>
        <ok to="hive-node"/>
        <error to="fail"/>
    </action>
  
    <action name="hive-node">
        <hive xmlns="uri:oozie:hive-action:0.2">
            <job-tracker>${jobTracker}</job-tracker>
            <name-node>${nameNode}</name-node>
            <prepare>
                <delete path="${nameNode}/user/maria_dev/oldmovies"/>
            </prepare>
            <configuration>
                <property>
                    <name>mapred.job.queue.name</name>
                    <value>${queueName}</value>
                </property>
            </configuration>
            <script>oldmovies.sql</script>
            <param>OUTPUT=/user/maria_dev/oldmovies</param>
        </hive>
        <ok to="end"/>
        <error to="fail"/>
    </action>
 
    <kill name="fail">
        <message>Sqoop failed, error message[${wf:errorMessage(wf:lastErrorNode())}]</message>
    </kill>
    <end name="end"/>
</workflow-app>


In [ ]:
#below is the properties file:

nameNode=hdfs://sandbox-hdp.hortonworks.com:8020
jobTracker=http://sandbox-hdp.hortonworks.com:8032
queueName=default 
oozie.use.system.libpath=true
oozie.wf.application.path=${nameNode}/user/maria_dev


Now upload our scripts and files into hdfs:
```
hadoop fs -put workflow.xml /user/maria_dev
hadoop fs -put oldmovies.sql /user/maria_dev
```
need to install a mySQL connector that oozie can see for sqoop to connect to MySQL:
```
hadoop fs -put /usr/share/java/mysql-connector-java.jar /user/oozie/share/lib/lib_20161025075203/sqoop
```
Then make sure to restart Oozie, and run:
```
oozie job -oozie http://localhost:11000/oozie -config /home/maria_dev/job.properties -run
```
Then you can go to http://127.0.0.1:11000/oozie to monitor the progress